In [ ]:
#载入数据
import pandas as pd
from sklearn.utils import shuffle
data_frame=pd.read_excel("total.xlsx")
df = data_frame.T #转置矩阵，因为dataFrame中每个结构的数据是以列排布的
dfs = shuffle(df.iloc[1:,:]) #对数据进行打乱，同时去除第一行
dfs = dfs.reset_index(drop=True) #重新设置索引

In [ ]:
#划分训练集与测试集
x_train = dfs.iloc[1:2197,:30263]
x_test = dfs.iloc[2197:,:30263]
y_train = dfs.iloc[1:2197,30264]
y_test = dfs.iloc[2197:,30264]
print("Size of training set:{} size of testing set:{}".format(x_train.shape[0],x_test.shape[0]))

# 对目标进行归一化 y = (y'-min(y'))/(max(y')-min(y'))
a = max(y_train)
b = min(y_train)
delta = a-b
y_train2 = (y_train-b)/delta
y_test2 = (y_test-b)/delta

# grid search
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
best_score = 0
best_parameters = {'solver':'lbfgs','hidden_layer_sizes':(100,100),'activation':'identity'}
for hid_lay in [(100,100),(10,10),(100,100,100),(10,10,10),(100,100,100,100),(10,10,10,10)]:
    for activate in ['identity','logistic','tanh','relu']:
        for sover in ['lbfgs','sgd','adam']:
            mlp = MLPRegressor(solver=sover,hidden_layer_sizes=hid_lay,activation=activate,alpha=0.01)#对于每种参数可能的组合，进行一次训练；
            scores = cross_val_score(mlp,x_train,y_train2,cv=5,scoring = 'r2')
            score = scores.mean() #取平均数
            print('solver: ',sover.rjust(5),'\n','hidden_layer_sizes: ',hid_lay,'\n','activation:',activate.rjust(9),'\n','score:',score,'\n','\n')
            if score > best_score:#找到表现最好的参数
                best_score = score
                best_parameters = {'solver':sover,'hidden_layer_sizes':hid_lay,'activation':activate}

mlp = MLPRegressor(**best_parameters,alpha=0.01)
mlp.fit(x_train,y_train2)
test_score = mlp.score(x_test,y_test2)
print("Best score on validation set:{:.2f}".format(best_score))
print("Best parameters:{}".format(best_parameters))
print("Score on testing set:{:.2f}".format(test_score))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA

#基于随机森林的预测
best_score2 = 0 # grid search
best_parameters2 = {'max_depth':2,'max_features':3,'min_samples_leaf':2} # 优化算法（solver），隐含层结构（hidden_layer_sizes），激活函数（activation）
for i in [2,3,4]:
    for j in [2,3,4]:
        for k in [2,3,4]:
            regr = RandomForestRegressor(max_depth=i,max_features=j,min_samples_leaf=k)
            scores = cross_val_score(regr,x_train,y_train,cv=5,scoring = 'r2')# ‘cv=5’表示进行五折交叉验证，scoring = 'r2'表示以R2作为评估参数，评估参数rmse,mse
            score = scores.mean() #取平均数
            print('max_depth: ',i,'\n','max_features: ',j,'\n','min_samples_leaf:',k,'\n','score:',score,'\n','\n')
            if score > best_score2:#找到表现最好的参数
                best_score2 = score
                best_parameters2 = {'max_depth':i,'max_features':j,'min_samples_leaf':k}



regr = RandomForestRegressor(**best_parameters2)
# regr = RandomForestRegressor(random_state=100,
#                              bootstrap=True,
#                              max_depth=2,
#                              max_features=2,
#                              min_samples_leaf=3,
#                              min_samples_split=5,
#                              n_estimators=3)
regr.fit(x_train, y_train)
y_pre2 = regr.predict(x_test)
test_score = regr.score(x_test,y_test)
print("Best score on validation set:{:.2f}".format(best_score2))
print("Best parameters:{}".format(best_parameters2))
print("Score on testing set:{:.2f}".format(test_score))

In [ ]:
import matplotlib.pyplot as plt
xx=range(0,len(y_test2))
pred_y = mlp.predict(x_test)
plt.figure(figsize=(18,6))
plt.scatter(xx,y_test2,color="red",label="Sample Point",linewidth=3) 
plt.plot(xx,pred_y,color="orange",label="Fitting Line",linewidth=2)
plt.legend()
plt.show()

In [1]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
# 模型拟合数据
clf.fit(x_train,y_train2).astype()
feat_importances = pd.Series(clf.feature_importances_, index=x_train.columns)
# 筛选出特征的重要性程度最大的10个特征
feat_importances.nlargest(10).plot(kind='barh', figsize = (8, 6))

NameError: name 'x_train' is not defined

In [ ]:
from sklearn.datasets import make_friedman1
# from sklearn.ensemble import GradientBoostingRegressor
from sklearn.inspection import plot_partial_dependence

# clf = GradientBoostingRegressor(n_estimators=10).fit(X, y)
plot_partial_dependence(mlp, features=[0, 2],X=df.iloc[1:,:10799],grid_resolution=10)  

plt.show()

In [ ]:
print(selectors.generation_scores_)

In [ ]:
x_rf_train = x_train.values
y_rf_train = y_train2.values
x_rf_test = x_test.values
y_rf_test = y_test2.values

In [ ]:
# 训练随机森林解决回归问题
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators=200, random_state=0)
regressor.fit(x_rf_train, y_rf_train)
y_pred = regressor.predict(x_rf_test)

# 评估回归性能
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_rf_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_rf_test, y_pred))
print('Root Mean Squared Error:',
      np.sqrt(metrics.mean_squared_error(y_rf_test, y_pred)))

In [ ]:
metrics.r2_score(y_rf_test, y_pred)

In [ ]:
import sklearn.datasets as datasets
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA


regr = RandomForestRegressor()
# regr = RandomForestRegressor(random_state=100,
#                              bootstrap=True,
#                              max_depth=2,
#                              max_features=2,
#                              min_samples_leaf=3,
#                              min_samples_split=5,
#                              n_estimators=3)
pipe = Pipeline([('scaler', StandardScaler()), ('reduce_dim', PCA()),
                 ('regressor', regr)])
pipe.fit(x_rf_train, y_rf_train)
ypipe = pipe.predict(x_rf_test)

from six import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz
import pydotplus
import os

# 执行一次
# os.environ['PATH'] = os.environ['PATH']+';'+r"D:\CLibrary\Graphviz2.44.1\bin\graphviz"
dot_data = StringIO()
export_graphviz(pipe.named_steps['regressor'].estimators_[0],
                out_file=dot_data)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
graph.write_png('tree.png')
Image(graph.create_png())

In [ ]:
#feature importance
from sklearn.ensemble import ExtraTreesClassifier

importances = regressor.feature_importances_
std = np.std([tree.feature_importances_ for tree in regressor.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
print("Feature ranking:")

for f in range(x_rf_train.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(x_rf_train.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(x_rf_train.shape[1]), indices)
plt.xlim([-1, x_rf_train.shape[1]])
plt.show()

In [ ]:
import shap
explainer = shap.TreeExplainer(regressor) #这里的model在准备工作中已经完成建模，模型名称就是model
shap_values = explainer.shap_values(x_rf_test) #这里的test_data是我的测试集，predictors是X_train的变量

In [ ]:
shap.summary_plot(shap_values, x_rf_test)